In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from fastai.vision import *
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
from helpers import *
from fastai.vision import *
import os
import fastai
import torch
from pathlib import Path
from fastai.callbacks.hooks import *
from PIL import ImageFilter
import numpy
from torchvision import transforms
import re
from PIL import Image as Image1
from fastai.vision import Image
from torchsummary1 import summary

In [ ]:
plt.rcParams['figure.figsize'] = (18,8)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 20}
plt.rc('font', **font)

# Loading the model

In [ ]:
def list_dir(self, pattern="*"):
    import glob
    return [Path(x) for x in glob.glob(str(self/pattern))]
Path.ls = list_dir

In [ ]:
def load_model():
    test_seed = 23 
    train_val_seed = 375
    path = Path('./scratch')
    folder='train2'
    data_dir = path/'cvs'
    file_format = 'cv-test_seed-{}-train_seed-{}.xlsx'
    data_file = data_dir / file_format.format(test_seed, train_val_seed)
    
    df_train_val, _ = nsea_load_data_from_excel(data_file)
    data=create_db_train_val(df_train_val, path, folder)
    model_name = "full-model-test_seed-{}-train_seed-{}".format(test_seed, train_val_seed)
    
    arch = models.resnet50

    learn = cnn_learner(data, arch)
    learn.load(model_name);
    return learn    

In [ ]:
learn = load_model()

In [ ]:
summary(learn.model, (3,576,704))

# Export Pickle

In [ ]:
learn.export(file = 'export_resnet50.pkl')

# Blurring the video frames

In [ ]:
def blur_image(img):
    #img = transforms.ToPILImage()(img.data).convert("RGB")
    img = Image1.fromarray(img)
    
    cropped = img.crop((10,60,130,200)) # left, up , right, bottom
    blur = cropped.filter(ImageFilter.GaussianBlur(radius=5))
    img.paste(blur, (10,60,130,200))
    
    cropped = img.crop((480,20,700,100))
    blur = cropped.filter(ImageFilter.GaussianBlur(radius=5))
    img.paste(blur, (480,20,700,100))
    
    cropped = img.crop((350,10,500,60))
    blur = cropped.filter(ImageFilter.GaussianBlur(radius=5))
    img.paste(blur, (350,10,500,60))
    
    cropped = img.crop((520,520,700,576))
    blur = cropped.filter(ImageFilter.GaussianBlur(radius=5))
    img.paste(blur, (520,520,700,576))
    
    cropped = img.crop((5,10,220,100))
    blur = cropped.filter(ImageFilter.GaussianBlur(radius=5))
    img.paste(blur, (5,10,220,100))
    
    #img = pil2tensor(img,np.float32)
    #img = img/255
    return img

# Thresholds

In [ ]:
pkl_file = open('optimal-thresholds.pkl', 'rb')
thresholds= pickle.load(pkl_file)
pkl_file.close()
thresholds

In [ ]:
thresholds['Anode']

# Real Time Rolling Prediction Average

In [ ]:
video_path = '/media/data/astamoulakatos/Survey-1-2016/DATA_20160823051752172/20160823051752484@DVR1_Ch2.asf'

In [ ]:
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(video_path)
# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

In [ ]:
classes = ['Anode', 'Burial', 'Exposure', 'Field Joint', 'Free Span']
i_list = []
cl1 = []
cl2 = []
cl3 = []
cl4 = []
cl5 = []

In [ ]:
# Read until video is completed
total_frames = 500
frame_counter = 0
while(cap.isOpened()):
  # Capture frame-by-frame
    ret, frame = cap.read()
    frame_counter = frame_counter + 1
    frame = blur_image(frame)
    frame = Image(pil2tensor(frame, dtype=np.float32).div_(255))
    if ret == True:
        # Make the prediction
        p = learn.predict(frame, thresh=0.5)
        classf = p[2].numpy()
        i_list.append(frame_counter+1)
        cl1.append(classf[0])
        cl2.append(classf[1])
        cl3.append(classf[2])
        cl4.append(classf[3])
        cl5.append(classf[4])
        # Display the resulting frame
        # plt.imshow(frame) and plt.show()
    else: 
        break
    if frame_counter > total_frames:
        break
# When everything done, release the video capture object
cap.release()

In [ ]:
data = {'Anode':cl1, 'Burial':cl2, 'Exposure':cl3, 'Field Joint':cl4, 'Free Span':cl5} 
  
df = pd.DataFrame(data) 
df

In [ ]:
frame.show(figsize=(18,8))

# Setting the thresholds

In [ ]:
df.head()

In [ ]:
th = np.array([thresholds[key] for key in thresholds])
th

In [ ]:
pred = df.to_numpy()

In [ ]:
pred

In [ ]:
## Apply digitisation on the outputs
for idx, event_type in enumerate(classes):
    pred[:,idx] = np.where(pred[:,idx] >= thresholds[event_type], 1, 0)
pred

In [ ]:
pred[:,0]

In [ ]:
plt.plot(pred) and plt.show()

# Rolling Average Prediction
## Neighboring Window
### window size = k = 10

In [ ]:
pred_rlp = pred

In [ ]:
k = 10

In [ ]:
#amount_of_ones = 0
for l in range(0,5):
    amount_of_ones = 0
    counter = 0
    for i in pred[:,l]:
        amount_of_ones = amount_of_ones + i
        counter = counter + 1
        if (counter == k):
            pred_rlp[:,l] = (amount_of_ones > k/2)
            amount_of_ones = 0
            counter = 0
    print(amount_of_ones)
    #pred[:,l] = (amount_of_ones > len(pred[:,l])/2)

In [ ]:
plt.plot(pred_rlp) and plt.show()

# Rolling Average Prediction
## Sliding Window

In [ ]:
from itertools import islice

In [ ]:
def window(seq, n=10):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [ ]:
pred_rlp_sw = pred
pred_rlp_sw

In [ ]:
for l in range(0,5):
    amount_of_ones = 0
    counter = 0
    for res in window(pred[:,l] , n=k): 
        #print(res)
        for i in res:
            print(i)
            amount_of_ones = amount_of_ones + i
            counter = counter + 1
            if (counter == k):
                pred_rlp_sw[:,l] = (amount_of_ones > k/2)
                amount_of_ones = 0
                counter = 0
            #print(amount_of_ones)

In [ ]:
plt.plot(pred_rlp_sw) and plt.show()

# Doing the same with PyTorch

In [ ]:
net = learn.model

In [ ]:
net;

In [ ]:
transform = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [ ]:
fram = frame.data

In [ ]:
fram = transform(fram)

In [ ]:
fram = fram.unsqueeze(dim = 0)

In [ ]:
fram.shape

In [ ]:
fram = fram.cuda()

In [ ]:
with torch.no_grad():
    out = net(fram)

In [ ]:
net.eval;

In [ ]:
out1 = net(fram)

In [ ]:
out1.sigmoid()

In [ ]:
plt.hist(fram.flatten())

In [ ]:
net = net.eval()

In [ ]:
frame = (frame.data).unsqueeze(dim = 0)

In [ ]:
frame = frame.cuda()

In [ ]:
output = net(frame)

In [ ]:
output.sigmoid()

In [ ]:
frame.shape

In [ ]:
frame.shape

In [ ]:
img1 = (frame.data).squeeze()

In [ ]:
plt.imshow((img1).permute(1, 2, 0)) and plt.show()

In [ ]:
(learn.model(frame)).sigmoid()